In [1]:
import pandas as pd
import numpy as np
import subprocess
import os

gff3Cols=["seqid","source","type","start","end","score","strand","phase","attributes"]

In [2]:
assemblyReportFP="./prepAnnotation/GCF_000001405.38_GRCh38.p12_assembly_report.txt"
chrMapCols=["name","role","molecule","type","genbank","rel","refseq","unit","seqLen","ucsc"]
assemblyReport=pd.read_csv(assemblyReportFP,sep="\t",names=chrMapCols,comment="#")
assemblyReport["ucsc"] = np.where(assemblyReport["ucsc"]=="na","chr"+\
                                assemblyReport["molecule"].astype(str)+\
                                "_"+\
                                assemblyReport["genbank"].str.split(".",expand=True)[0]+\
                                "v"+\
                                assemblyReport["genbank"].str.split(".",expand=True)[1]+\
                                "_"+\
                                assemblyReport["role"].str.split("-",expand=True)[0],assemblyReport["ucsc"])
assemblyReport["ucsc"] = assemblyReport.apply(lambda row: row["ucsc"].replace("_novel","_alt"),axis=1)

In [3]:
ref_uc = dict(zip(assemblyReport.refseq, assemblyReport.ucsc))

In [4]:
df = pd.read_csv("./prepAnnotation/refseq_vs_gencode_intersection.prot_lnc.gff",sep="\t",names=gff3Cols)
assert (len(set(df["seqid"]) - set(assemblyReport["ucsc"])) == 0),"not all sequence ids from gff are in the assembly report"
df.head()

,seqid,source,type,start,end,score,strand,phase,attributes
0,chr7,HAVANA,transcript,127588345,127591705,.,+,.,ID=ENST00000000233.9;gene_id=ENSG00000004059.1...
1,chr7,HAVANA,exon,127588345,127588565,.,+,.,Parent=ENST00000000233.9
2,chr7,HAVANA,exon,127589083,127589163,.,+,.,Parent=ENST00000000233.9
3,chr7,HAVANA,exon,127589485,127589594,.,+,.,Parent=ENST00000000233.9
4,chr7,HAVANA,exon,127590066,127590137,.,+,.,Parent=ENST00000000233.9


In [5]:
# first need to separate primary scaffolds from alts
setChr=set(df['seqid'])
alts = set(df[df["seqid"].str.endswith("_alt")]["seqid"])
prims = ["chr1","chr2","chr3","chr4","chr5","chr6","chr7","chr8","chr9","chr10","chr11","chr12","chr13",
         "chr14","chr15","chr16","chr17","chr18","chr19","chr20","chr21","chr22","chrX","chrY","chrM"]
rest = setChr - (alts.union(set(prims)))

In [6]:
subdf = df[df["seqid"].isin(set(prims).union(set(rest)))].reset_index(drop=True)
subdf.to_csv("./prepAnnotation/prims_and_rest.gff",sep='\t',index=False,header=False)

In [103]:
list(alts)

['chr6_GL000250v2_alt',
 'chr11_KI270829v1_alt',
 'chr19_KI270867v1_alt',
 'chr16_GL383557v1_alt',
 'chr17_GL383564v2_alt',
 'chr11_KI270831v1_alt',
 'chr8_KI270813v1_alt',
 'chr3_KI270781v1_alt',
 'chr6_GL000254v2_alt',
 'chr19_GL949749v2_alt',
 'chr19_KI270891v1_alt',
 'chr19_KV575255v1_alt',
 'chr6_KI270801v1_alt',
 'chr8_KI270814v1_alt',
 'chr4_KI270786v1_alt',
 'chr19_KI270917v1_alt',
 'chr22_KI270876v1_alt',
 'chr19_KI270866v1_alt',
 'chr13_KI270838v1_alt',
 'chr8_KI270819v1_alt',
 'chr11_KI270927v1_alt',
 'chr19_KI270865v1_alt',
 'chr8_KI270815v1_alt',
 'chr22_KI270879v1_alt',
 'chr19_KV575253v1_alt',
 'chr11_KZ559111v1_alt',
 'chr21_KI270872v1_alt',
 'chr19_KI270890v1_alt',
 'chr7_KI270809v1_alt',
 'chr19_KV575252v1_alt',
 'chr19_KI270919v1_alt',
 'chr6_KI270798v1_alt',
 'chr3_KZ559103v1_alt',
 'chr6_KI270797v1_alt',
 'chr7_GL383534v2_alt',
 'chr12_KI270835v1_alt',
 'chr7_KZ208913v1_alt',
 'chr19_KI270888v1_alt',
 'chr2_KI270776v1_alt',
 'chr1_KI270763v1_alt',
 'chr19_KV575247v

In [104]:
# now need to extract primary chromsomes and rests from the reference genome and build indices
baseDir = "./prepAnnotation/chroms/"

if not os.path.exists(baseDir):
    os.mkdir(baseDir)

outFP = None # is set when the input is being read to write a separate file per each desired chromosome

# initialize the altFile
altFP = "./prepAnnotation/chroms/alts.fa"
testFP = open(altFP,"w+")
testFP.close()

non_alts = []

skip = False # to skip a record or not

with open("./prepAnnotation/GCF_000001405.38_GRCh38.p12_genomic.fna","r") as refFP:
    for line in refFP.readlines():
        if line[0]==">":
            skip=False
            cur_refseq = line[1:].split(" ")[0]
            assert(cur_refseq in set(assemblyReport["refseq"])),cur_refseq+str(" not found in assembly report")
            cur_ucsc = ref_uc[cur_refseq]
            if cur_ucsc == "na": # skip the record since not known in ucsc
                skip=True
                continue
            else:
                if cur_ucsc in rest or cur_ucsc in prims: # proceed with rest and prims
                    non_alts.append(cur_ucsc)
                    if not outFP is None:
                        outFP.close()
                    outFP = open(baseDir+cur_ucsc+".fa","w+")
                    outFP.write(">"+str(cur_ucsc)+"\n")
                    
                else: # must be an alt
                    print(cur_ucsc)
                    if not outFP is None:
                        outFP.close()
                    outFP = open(altFP,"a")
                    outFP.write(">"+str(cur_ucsc)+"\n")
                    
        else:
            if not skip: # skip flag not activated - may continue safely
                outFP.write(line)
outFP.close()

chr1_KI270706v1_random
chr1_KI270707v1_random
chr1_KI270708v1_random
chr1_KI270709v1_random
chr1_KI270710v1_random
chr1_KI270711v1_random
chr1_KI270712v1_random
chr1_KI270714v1_random
chr2_KI270715v1_random
chr2_KI270716v1_random
chr3_GL000221v1_random
chr4_GL000008v2_random
chr5_GL000208v1_random
chr9_KI270717v1_random
chr9_KI270718v1_random
chr9_KI270719v1_random
chr9_KI270720v1_random
chr14_GL000009v2_random
chr14_GL000225v1_random
chr14_KI270722v1_random
chr14_GL000194v1_random
chr14_KI270723v1_random
chr14_KI270724v1_random
chr14_KI270725v1_random
chr16_KI270728v1_random
chr17_GL000205v2_random
chr17_KI270729v1_random
chr17_KI270730v1_random
chr22_KI270731v1_random
chr22_KI270732v1_random
chr22_KI270733v1_random
chr22_KI270735v1_random
chr22_KI270736v1_random
chr22_KI270737v1_random
chr22_KI270738v1_random
chr22_KI270739v1_random
chrY_KI270740v1_random
chrUn_KI270302v1
chrUn_KI270304v1
chrUn_KI270303v1
chrUn_KI270305v1
chrUn_KI270322v1
chrUn_KI270320v1
chrUn_KI270310v1
chrUn_KI270

chr21_GL383580v2_alt
chr21_GL383581v2_alt
chr21_KI270872v1_alt
chr22_KI270875v1_alt
chr22_KI270878v1_alt
chr22_KI270879v1_alt
chr22_KI270876v1_alt
chr22_KI270877v1_alt
chr22_GL383583v2_alt
chr22_GL383582v2_alt
chrX_KI270880v1_alt
chrX_KI270881v1_alt
chr1_KI270892v1_alt
chr2_KI270894v1_alt
chr2_KI270893v1_alt
chr3_KI270895v1_alt
chr4_KI270896v1_alt
chr5_KI270897v1_alt
chr5_KI270898v1_alt
chr6_GL000251v2_alt
chr7_KI270899v1_alt
chr8_KI270901v1_alt
chr8_KI270900v1_alt
chr11_KI270902v1_alt
chr11_KI270903v1_alt
chr12_KI270904v1_alt
chr15_KI270906v1_alt
chr15_KI270905v1_alt
chr17_KI270907v1_alt
chr17_KI270910v1_alt
chr17_KI270909v1_alt
chr17_JH159148v1_alt
chr17_KI270908v1_alt
chr18_KI270912v1_alt
chr18_KI270911v1_alt
chr19_GL949747v2_alt
chr22_KB663609v1_alt
chrX_KI270913v1_alt
chr3_KI270924v1_alt
chr4_KI270925v1_alt
chr6_GL000252v2_alt
chr8_KI270926v1_alt
chr11_KI270927v1_alt
chr19_GL949748v2_alt
chr22_KI270928v1_alt
chr3_KI270934v1_alt
chr6_GL000253v2_alt
chr19_GL949749v2_alt
chr3_KI27093

In [79]:
# now create gmap indices

k=10
q=2

non_alts = 

gmapIdxDir = "./prepAnnotation/gmapIDXs/"

if not os.path.exists(gmapIdxDir):
    os.mkdir(gmapIdxDir)
    
for cur_ucsc in non_alts:
    curIdxDir=gmapIdxDir+"/"+cur_ucsc+"/"
    if not os.path.exists(curIdxDir):
        os.mkdir(curIdxDir)
    
    chromFaFP=baseDir+cur_ucsc+".fa"
    assert os.path.exists(chromFaFP),"fasta file for chromosome "+cur_ucsc+" does not exist"
    
    # build gmap index
    subprocess.call(["gmap_build","--dir="+curIdxDir,"--db="+cur_ucsc,"-k",str(k),"-q",str(q),chromFaFP])

In [108]:
# first we need to separte all groups of alts from GFFs

gffDir = "./prepAnnotation/gffs/"

if not os.path.exists(gffDir):
    os.mkdir(gffDir)

prim_alts = list(set([x.split("_")[0] for x in alts]))
for pa in prim_alts: 
    # for each corresponding primary scaffold identify the group of alts and create a single df
    sub_alts = [x for x in alts if pa in x]
    subdf = df[df["seqid"].isin(sub_alts)].reset_index(drop=True)
    subdf.to_csv(gffDir+pa+".gff",sep='\t',index=False,header=False)

In [ ]:
# now we begin the alignment stage by running the script - alignTrans_parallel.py